In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load YOLO model
config_path = "yolov3.cfg"
weights_path = "yolov3.weights"
names_path = "coco.names"

# Load class labels
with open(names_path, "r") as f:
    classes = f.read().strip().split("\n")

# Load network
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Read image
image_path = "baggage_claim.jpg"  # Change to your image path
image = cv2.imread(image_path)
height, width = image.shape[:2]

# Convert image to YOLO input format
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)

# Get YOLO output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Forward pass
outputs = net.forward(output_layers)


# Withou

In [ ]:
# Extract bounding boxes, confidence scores, and class IDs
boxes = []
confidences = []
class_ids = []

confidence_threshold = 0.5  # Minimum confidence to consider a detection

for output in outputs:
    for detection in output:
        scores = detection[5:]  # Confidence scores for each class
        class_id = np.argmax(scores)  # Get the class with highest confidence
        confidence = scores[class_id]

        if confidence > confidence_threshold:
            center_x, center_y, w, h = (detection[:4] * [width, height, width, height]).astype("int")

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, int(w), int(h)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Draw detections *without* NMS
image_without_nms = image.copy()
for i in range(len(boxes)):
    x, y, w, h = boxes[i]
    color = (0, 255, 0)  # Green box
    cv2.rectangle(image_without_nms, (x, y), (x + w, y + h), color, 2)
    cv2.putText(image_without_nms, f"{classes[class_ids[i]]}: {confidences[i]:.2f}", 
                (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Display image
plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(image_without_nms, cv2.COLOR_BGR2RGB))
plt.title("Object Detection Without NMS (Overlapping Boxes)")
plt.axis("off")
plt.show()


In [ ]:
# Apply Non-Maximum Suppression
iou_threshold = 0.4  # IoU threshold for NMS
indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, iou_threshold)

# Draw final detections after NMS
image_with_nms = image.copy()
if len(indices) > 0:
    for i in indices.flatten():
        x, y, w, h = boxes[i]
        color = (255, 0, 0)  # Blue box
        cv2.rectangle(image_with_nms, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image_with_nms, f"{classes[class_ids[i]]}: {confidences[i]:.2f}", 
                    (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Display image
plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(image_with_nms, cv2.COLOR_BGR2RGB))
plt.title("Object Detection With NMS (Cleaned Detections)")
plt.axis("off")
plt.show()
